## CS3244 Project - Birdwatching SG

### Quick Setup
install the Kaggle API by uncommenting the following line and executing it.

In [ ]:
# ! pip install kaggle --upgrade

Then you need to upload your credentials from Kaggle on your instance. Login to kaggle and click on your profile picture on the top left corner, then 'My account'. Scroll down until you find a button named 'Create New API Token' and click on it. This will trigger the download of a file named 'kaggle.json'.

Upload this file to the directory this notebook is running in, by clicking "Upload" on your main Jupyter page, then uncomment and execute the next two commands (or run them in a terminal). For Windows, uncomment the last two commands.

In [ ]:
# ! mkdir -p ~/.kaggle/
# ! mv kaggle.json ~/.kaggle/

# For Windows, uncomment these two commands
# ! mkdir %userprofile%\.kaggle
# ! move kaggle.json %userprofile%\.kaggle

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.vision import *
bs = 64

### 1) Find All Birds 
Single shot multi-box detection (SSD) 

### 2) Export Single Bird Images

### 3) Bird Classification

In [ ]:
path = Config.data_path()/'birds'
path.mkdir(parents=True, exist_ok=True)
path

In [ ]:
# ! kaggle datasets download hsankesara/cub200
# ! unzip -q -n {path}/train_v2.csv.zip -d {path}